# The Battle of the Neighborhoods Report

## Introduction/ Business Problem

My client is a small business owner from New York and is trying to open a second location of their business in the city of Toronto. The business is a family-owned Italian restaurant. In the past, my client's biggest struggle has been completing with other Italian restaurants. Therefore, I will recommend a location for my client based on the concentration of my client's competitors in a neighborhood. 

## Data and Method

I will be using Foursquare API Data to assess the Toronto. The data points that I will be using are the locations of Italian restaurants. Based on the concentration of these locations I will recommend a location for my client. My recommendation will most likely be in a location where there is a low concentration of these restaurants, but there are still some present. The low concentration is important because it means that the competition will be less. The presence of other Italian restaurants is also important in establishing my client's new business location because it means that the demographic of that neighborhood probably enjoys Italian food and will become patrons of my client. 

#### Import Libraries

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [6]:
from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors
 
from sklearn.cluster import KMeans
import folium 

#### Define Foursquare Credentials 

In [7]:
CLIENT_ID = 'EZJRMM13FKSKWYDHDPRC23MLN2QYZQT4BAAFP3E2KDJG2DI5' # your Foursquare ID
CLIENT_SECRET = 'LQF4QP2QVLS5KL1D53GXPQT3OGHTTRTH4XWLYKKFHUALCDBO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EZJRMM13FKSKWYDHDPRC23MLN2QYZQT4BAAFP3E2KDJG2DI5
CLIENT_SECRET:LQF4QP2QVLS5KL1D53GXPQT3OGHTTRTH4XWLYKKFHUALCDBO


#### Search and Clean Dataframe

In [8]:
search_query = 'Italian'
radius = 10000
print(search_query + ' .... OK!')

Italian .... OK!


In [9]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


In [10]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, 43.653963, -79.387207, VERSION, search_query, radius, LIMIT)

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d4d973286bc490032b3caa5'},
 'response': {'venues': [{'id': '52f6816f11d24a43115dc834',
    'name': 'Scaddabush Italian Kitchen & Bar',
    'location': {'address': '382 Yonge Street, Unit #7',
     'crossStreet': 'Gerrard',
     'lat': 43.658920292028725,
     'lng': -79.38289105381784,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.658920292028725,
       'lng': -79.38289105381784}],
     'distance': 652,
     'postalCode': 'M5B 1S8',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['382 Yonge Street, Unit #7 (Gerrard)',
      'Toronto ON M5B 1S8',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'shortName': 'Italian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId

In [12]:
venues = results['response']['venues'] 

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,52f6816f11d24a43115dc834,"382 Yonge Street, Unit #7",CA,Toronto,Canada,Gerrard,652,"[382 Yonge Street, Unit #7 (Gerrard), Toronto ...","[{'label': 'display', 'lat': 43.65892029202872...",43.658920,-79.382891,M5B 1S8,ON,Scaddabush Italian Kitchen & Bar,v-1565366066,NaN
1,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,5b897e92db1d81002c91df8c,66 Wellington St W,CA,Toronto,Canada,NaN,877,"[66 Wellington St W, Toronto ON M5K 1E7, Canada]","[{'label': 'display', 'lat': 43.647161, 'lng':...",43.647161,-79.381691,M5K 1E7,ON,Fabbrica Rustic Italian,v-1565366066,NaN
2,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",False,4e31afdd091a973ec9c5a2b5,146 Cumberland St,CA,Toronto,Canada,btwn Avenue Rd & Bay St,1832,"[146 Cumberland St (btwn Avenue Rd & Bay St), ...","[{'label': 'display', 'lat': 43.66995452843031...",43.669955,-79.392603,M5R 1A8,ON,"Punto Gelato, Simply Italian",v-1565366066,NaN
3,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,581cad6a7c74e15859a6f890,"200 Front Street W., Unit #G001",CA,Toronto,Canada,at Simcoe St,1037,"[200 Front Street W., Unit #G001 (at Simcoe St...","[{'label': 'display', 'lat': 43.6447367776608,...",43.644737,-79.385355,M5V 3J1,ON,Scaddabush Italian Kitchen & Bar,v-1565366066,NaN
4,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",False,4bfc0289c3ba9521c00f9653,136 Beverley St,CA,Toronto,Canada,Dundas Street,555,"[136 Beverley St (Dundas Street), Toronto ON, ...","[{'label': 'display', 'lat': 43.65402694219784...",43.654027,-79.394104,NaN,ON,Italian Consulate Toronto,v-1565366066,NaN


In [17]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.dropna(inplace = True)
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Scaddabush Italian Kitchen & Bar,Italian Restaurant,"382 Yonge Street, Unit #7",CA,Toronto,Canada,Gerrard,652,"[382 Yonge Street, Unit #7 (Gerrard), Toronto ...","[{'label': 'display', 'lat': 43.65892029202872...",43.658920,-79.382891,M5B 1S8,ON,52f6816f11d24a43115dc834
2,"Punto Gelato, Simply Italian",Ice Cream Shop,146 Cumberland St,CA,Toronto,Canada,btwn Avenue Rd & Bay St,1832,"[146 Cumberland St (btwn Avenue Rd & Bay St), ...","[{'label': 'display', 'lat': 43.66995452843031...",43.669955,-79.392603,M5R 1A8,ON,4e31afdd091a973ec9c5a2b5
3,Scaddabush Italian Kitchen & Bar,Italian Restaurant,"200 Front Street W., Unit #G001",CA,Toronto,Canada,at Simcoe St,1037,"[200 Front Street W., Unit #G001 (at Simcoe St...","[{'label': 'display', 'lat': 43.6447367776608,...",43.644737,-79.385355,M5V 3J1,ON,581cad6a7c74e15859a6f890
6,Jamie's Italian,Italian Restaurant,3401 Dufferin St,CA,Toronto,Canada,Allen Road and 401,9679,"[3401 Dufferin St (Allen Road and 401), Toront...","[{'label': 'display', 'lat': 43.72668644119483...",43.726686,-79.453133,M6A 2T9,ON,566344c9498eedf4e11af0fa
7,Kit Kat Italian Bar & Grill,Italian Restaurant,297 King St W,CA,Toronto,Canada,at John St,870,"[297 King St W (at John St), Toronto ON M5V 1J...","[{'label': 'display', 'lat': 43.64641598988062...",43.646416,-79.390030,M5V 1J5,ON,4b3ace79f964a520ae6e25e3
8,The Fresh Italian Eatery,Italian Restaurant,"109 McCaul Street, Unit #42",CA,Toronto,Canada,Dundas Street West,288,"[109 McCaul Street, Unit #42 (Dundas Street We...","[{'label': 'display', 'lat': 43.653889, 'lng':...",43.653889,-79.390785,M5T 3K5,ON,526fe29411d2aeb3803013b0
9,Mustachio Italian Eatery,Italian Restaurant,595 Bay St,CA,Toronto,Canada,Dundas St,405,"[595 Bay St (Dundas St), Toronto ON M5G 2C2, C...","[{'label': 'display', 'lat': 43.65616, 'lng': ...",43.656160,-79.383190,M5G 2C2,ON,573df789498e03dd8e54b166
25,Little Anthony's,Italian Restaurant,121 Richmond St. W,CA,Toronto,Canada,at York St.,462,"[121 Richmond St. W (at York St.), Toronto ON ...","[{'label': 'display', 'lat': 43.65029624519052...",43.650296,-79.384513,M5H 2K1,ON,4b846dd4f964a520dc3431e3
27,Sbarro,Pizza Place,220 Yonge St.,CA,Toronto,Canada,in Toronto Eaton Centre,526,"[220 Yonge St. (in Toronto Eaton Centre), Toro...","[{'label': 'display', 'lat': 43.655518, 'lng':...",43.655518,-79.381030,M5B 2H1,ON,4b4a2d09f964a520687d26e3
29,Marinella Simply Italian,Italian Restaurant,640 College St.,CA,Toronto,Canada,College and Grace,2304,"[640 College St. (College and Grace), Toronto ...","[{'label': 'display', 'lat': 43.65502851758536...",43.655029,-79.415784,M6G 1B4,ON,4ba6adeef964a520546839e3


In [26]:
dataframe_filtered.dtypes

name                 object
categories           object
address              object
cc                   object
city                 object
country              object
crossStreet          object
distance              int64
formattedAddress     object
labeledLatLngs       object
lat                 float64
lng                 float64
postalCode           object
state                object
id                   object
dtype: object

In [74]:
dataframe_filtered.astype({'name':'str'})
df = dataframe_filtered.drop(['id', 'categories', 'address', 'cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress', 'labeledLatLngs', 'state'], axis = 1)
df.head()

,name,lat,lng,postalCode
0,Scaddabush Italian Kitchen & Bar,43.658920,-79.382891,M5B 1S8
2,"Punto Gelato, Simply Italian",43.669955,-79.392603,M5R 1A8
3,Scaddabush Italian Kitchen & Bar,43.644737,-79.385355,M5V 3J1
6,Jamie's Italian,43.726686,-79.453133,M6A 2T9
7,Kit Kat Italian Bar & Grill,43.646416,-79.390030,M5V 1J5


#### Visualize Data 

In [75]:
dataframe_filtered.name

0     Scaddabush Italian Kitchen & Bar
2         Punto Gelato, Simply Italian
3     Scaddabush Italian Kitchen & Bar
6                      Jamie's Italian
7          Kit Kat Italian Bar & Grill
8             The Fresh Italian Eatery
9             Mustachio Italian Eatery
25                    Little Anthony's
27                              Sbarro
29            Marinella Simply Italian
Name: name, dtype: object

In [92]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Toronto City Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

#### Perform a K-Means test to determine the region with the least concentration of Italian Restaurants 

In [87]:
kclus = 2

df_clus = df.drop('name', 1)
df_cluster = df_clus.drop('postalCode', 1)

kmeans = KMeans(n_clusters = kclus, random_state = 0).fit(df_cluster)

kmeans.labels_[1:10]

array([0, 0, 1, 0, 1, 1], dtype=int32)

In [88]:
cluster_labels = pd.DataFrame(kmeans.labels_)[0]
cluster_labels.head()

0    1
1    0
2    0
3    1
4    0
Name: 0, dtype: int32

In [89]:
df['Cluster'] = cluster_labels
df.dtypes


name           object
lat           float64
lng           float64
postalCode     object
Cluster       float64
dtype: object

In [90]:
df = pd.DataFrame(df)
df.dropna(inplace = True)
df = df.astype({'Cluster' : 'int32'})

In [93]:
map_clusters = folium.Map(location = [lat, lng], zoom_start = 10)

x = np.arange(kclus)
ys = [i + x + (i*x) ** 2 for i in range(kclus)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

marker_color = []
for lat, lng, poi, cluster in zip(df["lat"], df['lng'], df['name'], df['Cluster']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = rainbow [cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
    
map_clusters

### Results, Discussion, and Conclusion

From this study, I have concluded that my clinet should avoid the area between Blor Street West and Front Street West as there is a high concentration of Italian restaurants in that area. Additionally, due to the lack of data in the Foursquare API about Italian restaurants in Toronto, performing an accurate k-means test was futile. The locations of Italian restaurants do not need to be clustered in order assess their locations.

I recommend that my client place thier location near Jamie's Italian next to Yorkdale shopping center. Jamie's was clearly an outlier value, however I chose to include it so that there were more data points to work with. The presense of Jamie's so far away from the rest of the values represents that the demographic in the areas where there are no Italian restaurants probably will not go to an Italian resaurant near them. 

I should have performed a DBSCAN test instead as it would have ignored the noisy data. However, the lack of data points would have still resulted in a relatively poor outcome anyways.

All in all, my client should avoid the Downtown Toronoto area as they may struggle with the amount of competition in that region. Additionally, they should consider opening thier Italian restaurant in near the Yorkdale Shopping Center since there is already one Italian place, indicating that the demographic near there may appreciate Italian food. 